# Useful functions

In [1]:
import numpy as np
import pandas as pd 
import keras
import os
import re
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing, model_selection
from sklearn.datasets import make_moons
from sklearn.datasets import make_circles
from sklearn import datasets
from tensorflow.keras.datasets import mnist


Using TensorFlow backend.
/home/quentin/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/quentin/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/quentin/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/quentin/env/lib/python3.6/site-packages/tensorflow/python/framewo

In [2]:
def get_goodXy (X,y):
    ynew = model.predict_classes(X)
    X_good =[]
    y_good=[]
    for i in range(len(X)):
        if (ynew[i]==0 and y[i]==1) or (ynew[i]==1 and y[i]==0):
            print ("error prediction for X=%s, Predicted=%s, Real=%s"% (X[i], ynew[i], y[i]))
        else :
            X_good.append(X[i])
            y_good.append(y[i])
    return X_good,y_good        

In [3]:
def get_result_layers(model,X):
    result_layers=[]
    for i in range (len(model.layers)-1):
        hidden_layers= keras.backend.function(
                [model.layers[0].input],   
                [model.layers[i].output,] 
                )    
        result_layers.append(hidden_layers([X_good])[0])  
    return result_layers



In [4]:
def save_result_layers(filename,X,y,result_layers):
    f = open(filename, "w")
    for nb_X in range (len(X)):
        #my_string=""
        my_string=str(y[nb_X])+','
        for nb_layers in range (len(model.layers)-1):
            my_string+="<b>,"
            for j in range (len(result_layers[nb_layers][nb_X])):
                my_string+=str(result_layers[nb_layers][nb_X][j])+','
            my_string+="</b>,"    
        my_string=my_string [0:-1]
        my_string+='\n'
        f.write(my_string)    
    f.close()
    

## Iris dataset 

In [5]:
url="https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
names = ['SepalLengthCm', 'SepalWidthCm', 
         'PetalLengthCm', 'PetalWidthCm', 
         'Species']

data = pd.read_csv(url, names=names)

#Classification binaire sur Virginica et Setosa seulement
data=data[data['Species'].isin(['Iris-virginica', 'Iris-setosa'])]

i = 8
data_to_predict = data[:i].reset_index(drop = True)
predict_species = data_to_predict.Species 
predict_species = np.array(predict_species)
prediction = np.array(data_to_predict.drop(['Species'],axis= 1))

data = data[i:].reset_index(drop = True)

X = data.drop(['Species'], axis = 1)
X = np.array(X)
y = data['Species']

encoder = LabelEncoder()
y=encoder.fit_transform(y)


train_X, test_X, train_y, test_y = model_selection.train_test_split(X,y,test_size = 0.1, random_state = 0)




In [6]:
# Utilisation de keras comme classifieur
# mettre sigmoid comme fonction car binaire. Attention 1 seul neurone en sortie
input_dim = len(data.columns) - 1
model = Sequential()
model.add(Dense(8, input_dim = input_dim , activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(8, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )

model.fit(train_X, train_y, epochs = 10, batch_size = 2)

scores = model.evaluate(test_X, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/10
82/82 [==============================] - 1s 8ms/step - loss: 0.7044 - acc: 0.5610
Epoch 2/10
82/82 [==============================] - 0s 2ms/step - loss: 0.6161 - acc: 0.5610
Epoch 3/10
82/82 [==============================] - 0s 2ms/step - loss: 0.5516 - acc: 0.5610
Epoch 4/10
82/82 [==============================] - 0s 2ms/step - loss: 0.4480 - acc: 0.6829
Epoch 5/10
82/82 [==============================] - 0s 2ms/step - loss: 0.3425 - acc: 1.0000
Epoch 6/10
82/82 [==============================] - 0s 2ms/step - loss: 0.2679 - acc: 1.0000
Epoch 7/10
82/82 [==============================] - 0s 2ms/step - loss: 0.2079 - acc: 1.0000
Epoch 8/10
82/82 [==============================] - 0s 3ms/step - loss: 0.1552 - acc: 1.0000
Epoch 9/10
82/82 [==============================] - 0s 3ms/step - loss: 0.1105 - acc: 1.0000
Epoch 10/10
10/10 [==============================] - 0s 9ms/step

acc: 100.00%


In [7]:
# Récupération seulement des bons classés
X_good,y_good=get_goodXy (train_X, train_y)

In [8]:
# Récupération des valeurs de tous les layers sauf le dernier
result_layers=get_result_layers(model,X_good)

In [9]:
# Sauvegarde du fichier
# structure :
# 0/1 = valeur de la classe
# chaque valeur de layer est entourée par un []
save_result_layers("iris_8_10_8_tmp",X_good,y_good,result_layers)
# tri du fichier
os.system ('sort iris_8_10_8_tmp > iris_8_10_8_.csv')
# effacer le fichier intermédiaire
os.system ('rm iris_8_10_8_tmp')

0

## Makemoons dataset 

In [10]:
X, y = make_moons(n_samples=1000, noise=0.05, random_state=0)

validation_size=0.6 #40% du jeu de données pour le test

testsize= 1-validation_size
seed=30
# séparation jeu d'apprentissage et jeu de test
train_X, test_X, train_y, test_y=model_selection.train_test_split(X, 
                                               y, 
                                               train_size=validation_size, 
                                               random_state=seed,
                                               test_size=testsize)



In [11]:
# Utilisation de keras comme classifieur
# mettre sigmoid comme fonction car binaire. Attention 1 seul neurone en sortie
input_dim = 2

model = Sequential()
model.add(Dense(3, input_dim = input_dim , activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(3, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )

model.fit(train_X, train_y, epochs = 40, batch_size = 64)

scores = model.evaluate(test_X, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/40
600/600 [==============================] - 1s 1ms/step - loss: 0.6932 - acc: 0.5033
Epoch 2/40
600/600 [==============================] - 0s 75us/step - loss: 0.6931 - acc: 0.5033
Epoch 3/40
600/600 [==============================] - 0s 90us/step - loss: 0.6931 - acc: 0.5033
Epoch 4/40
600/600 [==============================] - 0s 85us/step - loss: 0.6931 - acc: 0.5033
Epoch 5/40
600/600 [==============================] - 0s 76us/step - loss: 0.6932 - acc: 0.5033
Epoch 6/40
600/600 [==============================] - 0s 80us/step - loss: 0.6931 - acc: 0.5033
Epoch 7/40
600/600 [==============================] - 0s 78us/step - loss: 0.6931 - acc: 0.5033
Epoch 8/40
600/600 [==============================] - 0s 82us/step - loss: 0.6931 - acc: 0.5033
Epoch 9/40
600/600 [==============================] - 0s 77us/step - loss: 0.6931 - acc: 0.5033
Epoch 10/40
600/600 [==============================] - 0s 77us/step - loss: 0.6931 - acc: 0.5033
Epoch 11/40
600/600 [===================

In [12]:
# Récupération seulement des bons classés
X_good,y_good=get_goodXy (train_X, train_y)

error prediction for X=[ 1.91258319 -0.02050477], Predicted=[0], Real=1
error prediction for X=[ 0.46386062 -0.34983752], Predicted=[0], Real=1
error prediction for X=[ 1.73868422 -0.23020397], Predicted=[0], Real=1
error prediction for X=[ 1.69941636 -0.0902471 ], Predicted=[0], Real=1
error prediction for X=[-0.02127692  0.43276319], Predicted=[0], Real=1
error prediction for X=[ 1.4440003  -0.27070936], Predicted=[0], Real=1
error prediction for X=[1.91498583 0.24413469], Predicted=[0], Real=1
error prediction for X=[1.95940778 0.41269847], Predicted=[0], Real=1
error prediction for X=[ 1.73674127 -0.18914707], Predicted=[0], Real=1
error prediction for X=[ 0.90164201 -0.49598578], Predicted=[0], Real=1
error prediction for X=[ 0.68827666 -0.42739017], Predicted=[0], Real=1
error prediction for X=[1.96794954 0.56330206], Predicted=[0], Real=1
error prediction for X=[1.87009386 0.02941778], Predicted=[0], Real=1
error prediction for X=[ 1.78022302 -0.14096716], Predicted=[0], Real=1


error prediction for X=[ 1.4432624 -0.403265 ], Predicted=[0], Real=1
error prediction for X=[1.95458913 0.12701995], Predicted=[0], Real=1
error prediction for X=[ 0.47224436 -0.28129086], Predicted=[0], Real=1
error prediction for X=[1.92971771 0.16493063], Predicted=[0], Real=1
error prediction for X=[0.06092146 0.14241808], Predicted=[0], Real=1
error prediction for X=[ 1.63401677 -0.33225386], Predicted=[0], Real=1
error prediction for X=[ 1.6782009  -0.20198687], Predicted=[0], Real=1
error prediction for X=[ 1.53671798 -0.38015053], Predicted=[0], Real=1
error prediction for X=[ 1.47391568 -0.33877942], Predicted=[0], Real=1
error prediction for X=[ 1.78819227 -0.12973943], Predicted=[0], Real=1
error prediction for X=[ 1.09174793 -0.46306249], Predicted=[0], Real=1
error prediction for X=[1.94401642 0.34854826], Predicted=[0], Real=1
error prediction for X=[ 0.27955131 -0.31834945], Predicted=[0], Real=1
error prediction for X=[-0.01289267  0.27685662], Predicted=[0], Real=1
er

error prediction for X=[ 1.67568975 -0.22968208], Predicted=[0], Real=1
error prediction for X=[ 0.8406083  -0.57464379], Predicted=[0], Real=1
error prediction for X=[ 0.32515039 -0.32838537], Predicted=[0], Real=1
error prediction for X=[ 1.38784935 -0.41236273], Predicted=[0], Real=1
error prediction for X=[1.98799225 0.414105  ], Predicted=[0], Real=1
error prediction for X=[ 0.4119216  -0.24592777], Predicted=[0], Real=1
error prediction for X=[ 0.40120659 -0.23351411], Predicted=[0], Real=1
error prediction for X=[2.01108954 0.44227458], Predicted=[0], Real=1
error prediction for X=[ 0.85102061 -0.50589137], Predicted=[0], Real=1
error prediction for X=[ 1.63320058 -0.22160531], Predicted=[0], Real=1
error prediction for X=[ 0.34888228 -0.18793122], Predicted=[0], Real=1
error prediction for X=[ 1.40984549 -0.48242883], Predicted=[0], Real=1
error prediction for X=[ 0.18790158 -0.12805293], Predicted=[0], Real=1


In [13]:
# Récupération des valeurs de tous les layers sauf le dernier
result_layers=get_result_layers(model,X_good)

In [14]:
# Sauvegarde du fichier
# structure :
# 0/1 = valeur de la classe
# chaque valeur de layer est entourée par une étoile *
save_result_layers("makemoons_3_10_10_3_tmp",X_good,y_good,result_layers)
# tri du fichier
os.system ('sort makemoons_3_10_10_3_tmp > makemoons_3_10_10_3_.csv')
# effacer le fichier intermédiaire
os.system ('rm makemoons_3_10_10_3_tmp')

0

## Mnist dataset 

In [15]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train_sample=X_train[0:100]
y_train_sample=y_train[0:100]

X_train=X_train_sample
y_train=y_train_sample
X_train = X_train.reshape(100, 784)
X_train = X_train.astype('float32')

X_train /= 255



X_01=[]
y_01=[]
nb_X=0
for i in range(X_train.shape[0]):
    if (y_train[i]==0 or y_train[i]==1):
        
        nb_X+=1
        X_01.append(X_train[i])
        y_01.append(y_train[i])

       
train_X=np.asarray(X_01)

train_y=y_01

encoder = LabelEncoder()
train_y=encoder.fit_transform(train_y)


In [16]:
input_dim = 784

model = Sequential()
model.add(Dense(512, input_dim = input_dim , activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )

model.fit(train_X, train_y, epochs = 40, batch_size = 32)



Epoch 1/40
27/27 [==============================] - 1s 22ms/step - loss: 0.7341 - acc: 0.3704
Epoch 2/40
27/27 [==============================] - 0s 1ms/step - loss: 0.4064 - acc: 0.8519
Epoch 3/40
27/27 [==============================] - 0s 1ms/step - loss: 0.2594 - acc: 1.0000
Epoch 4/40
27/27 [==============================] - 0s 2ms/step - loss: 0.1619 - acc: 1.0000
Epoch 5/40
27/27 [==============================] - 0s 255us/step - loss: 0.1022 - acc: 1.0000
Epoch 6/40
27/27 [==============================] - 0s 632us/step - loss: 0.0631 - acc: 1.0000
Epoch 7/40
27/27 [==============================] - 0s 371us/step - loss: 0.0406 - acc: 1.0000
Epoch 8/40
27/27 [==============================] - 0s 680us/step - loss: 0.0269 - acc: 1.0000
Epoch 9/40
27/27 [==============================] - 0s 567us/step - loss: 0.0186 - acc: 1.0000
Epoch 10/40
27/27 [==============================] - 0s 534us/step - loss: 0.0135 - acc: 1.0000
Epoch 11/40
27/27 [==============================] - 0s 

In [17]:
X_good,y_good=get_goodXy (train_X, train_y)

In [18]:
# Récupération des valeurs de tous les layers sauf le dernier
result_layers=get_result_layers(model,X_good)

In [19]:
# Sauvegarde du fichier
# structure :
# 0/1 = valeur de la classe
# chaque valeur de layer est entourée par une étoile *
save_result_layers("mnist_512_tmp",X_good,y_good,result_layers)
# tri du fichier
os.system ('sort mnist_512_tmp > mnist_512_.csv')
# effacer le fichier intermédiaire
os.system ('rm mnist_512_tmp')

0

# Extract Patterns

In [20]:
def get_directory_layers_from_csv(filename):
    tokens=filename.split("_")
    df = pd.read_csv(filename, sep = ',', header = None) 

    
    # creation d'un répertoire pour sauver tous les fichiers
    repertoire=filename[0:-4]
    os.makedirs(repertoire, exist_ok=True)
    string = repertoire+'/'+tokens[0]+'_'
    f=[]
    filenames=[]
    for nb_tokens in range (1,len(tokens)-1):
        name_file=string+'l'+str(nb_tokens)+'_'+tokens[nb_tokens]+'.csv'
        f.append(open(name_file, "w"))
        filenames.append(name_file)
        
        
    # sauvegarde du dataframe dans une chaîne de caracteres
    ch = df.to_string(header=False,
                  index=False,
                  index_names=False).split('\n')
    vals = [','.join(ele.split()) for ele in ch]
    
    # sauvegarde dans des fichiers spécifiques par layer
    token_layer=[]
    token_exemples=[]
    for nb_exemples in range (len(vals)):
        deb=str(df[0][nb_exemples])+','
        # 1 ligne correspond à une chaine
        s=vals[nb_exemples]
        listoftokens=re.findall(r'<b>,(.+?),</b>', s)
        nb_layers=len(listoftokens)
        
        for nb_token in range (nb_layers):
            save_token=''
            save_token=deb+str(listoftokens[nb_token])+'\n'
            
            f[nb_token].write(save_token)

    # sauvegarde d'un fichier qui contient tous les layers en une fois
    # récupération des données pour enlever les <b> et </b>
    df_all=pd.DataFrame()
    myindex=0
    for nb_columns in range(df.shape[1]):
        df[nb_columns]=df[nb_columns].astype(str)
        if (df[nb_columns]!='<b>').all() and (df[nb_columns]!='</b>').all():
            df_all[myindex]=df[nb_columns]
            myindex+=1

    # construction du nom du fichier de sauvegarde
    string = repertoire+'/'+tokens[0]+'_'
    for nb_tokens in range (1,len(tokens)-1):
        string+='l'+str(nb_tokens)+'_'+tokens[nb_tokens]+'_'
    string+='.csv'       
    # sauvegarde en .csv
    df_all.to_csv(string, sep=',', encoding='utf-8',index=False)

In [21]:
#Create a directory with a specific file for all the layers
filename="iris_8_10_8_.csv"    
get_directory_layers_from_csv(filename)    

filename='makemoons_3_10_10_3_.csv'
get_directory_layers_from_csv(filename) 

filename='mnist_512_.csv'
get_directory_layers_from_csv(filename) 

## Traitement

In [22]:
def discretise_dataset(filename,bins):
    df = pd.read_csv(filename, sep = ',', header = None) 
    oneColumn = np.array(df[1])
    for i in range(2,df.shape[1]):
        oneColumn=np.append(oneColumn,np.array(df[i]))
    dfoneColumn=pd.DataFrame(oneColumn)
    nb_bins=bins
    dftemp=pd.DataFrame()
    dftemp[0]=pd.cut(dfoneColumn[0], bins=nb_bins, labels=np.arange(nb_bins), right=False)
    df_new=pd.DataFrame(df[0])
    nb_tuples=df.shape[0]
    j=0
    for i in range(1,df.shape[1]):
        df_new[i]=np.copy(dftemp[0][j:j+nb_tuples])
        j+=nb_tuples
    return df_new
    
# test de la fonction    
df=discretise_dataset('iris_8_10_8_/iris_l1_8.csv',5)
print (df.head())
print (df.tail())

#Exemple d'utilisation de la mesure de levenstein
#pip install python-levenshtein
from Levenshtein import distance
print (distance('3,10,0,4,0,0,9,0', '4,8,0,2,0,0,8,0'))

#Creation d'une matrice de distance
df_mat=pd.DataFrame(X)
string="toto.csv"
df_mat.to_csv(string, sep=',', encoding='utf-8',index=False, header=False)
df_mat=pd.read_csv(string, sep = ',', header = None)
#recuperation de la matrice sous la forme de chaine
ch = df_mat.to_string(header=False,
                  index=False,
                  index_names=False).split('\n')
vals = [','.join(ele.split()) for ele in ch]
print (vals)    



List1 = vals
List2 = vals
Matrix = np.zeros((len(List1),len(List2)),dtype=np.int)

for i in range(0,len(List1)):
    for j in range(0,len(List2)):
        Matrix[i,j] = distance(List1[i],List2[j])

print (Matrix)

   0  1  2  3  4  5  6  7  8
0  0  0  0  0  0  0  2  0  0
1  0  0  0  0  0  0  2  0  0
2  0  0  0  0  0  0  2  0  0
3  0  0  0  0  0  0  2  0  0
4  0  0  0  0  0  0  2  0  0
    0  1  2  3  4  5  6  7  8
77  1  0  0  0  0  1  4  0  1
78  1  0  0  0  0  1  4  0  1
79  1  0  0  0  0  1  4  0  2
80  1  0  0  0  0  1  4  0  2
81  1  0  0  0  0  1  4  0  1
5
['2.021001,0.490179', '1.678201,-0.201987', '-0.282245,0.858785', '-0.021440,0.176281', '0.504842,-0.391043', '1.969539,0.360055', '0.956596,0.253665', '0.094879,0.983378', '-0.441660,0.872034', '0.707516,-0.512674', '-1.052250,0.047481', '0.073445,0.182330', '1.320594,-0.509063', '0.082926,0.916587', '0.103743,0.010696', '0.894168,0.341012', '1.817608,0.000681', '0.475219,0.819541', '1.027841,0.159873', '0.824209,0.480516', '0.900805,0.244685', '-0.970103,0.134214', '-0.456163,0.843592', '-0.371109,0.938706', '1.741274,-0.190401', '1.334030,-0.421556', '1.031463,0.221893', '0.869790,0.622999', '1.055784,-0.454860', '0.946276,0.043335',

[[ 0 11 14 ... 12 11 12]
 [11  0 13 ... 12 12 12]
 [14 13  0 ... 10 12 13]
 ...
 [12 12 10 ...  0 11 10]
 [11 12 12 ... 11  0 10]
 [12 12 13 ... 10 10  0]]
